<a href="https://colab.research.google.com/github/AmazingGrace-D/Text-Classification-with-RNN/blob/main/Text_Classification_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow_datasets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Bidirectional, LSTM, Embedding
import tensorflow_datasets as tfds

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2ANT68/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2ANT68/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2ANT68/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
dataset

{'test': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>}

In [ ]:
dataset['train']

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [ ]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
for example, label in train_dataset.take(3):
  print('Text:', example.numpy())
  print('Label:', label.numpy())

Text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Label: 0
Text: b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The

In [ ]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [ ]:
train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
# dataset = tf.data.Dataset.range(10)
# dataset = dataset.shuffle(3, reshuffle_each_iteration=False).batch(3).prefetch(tf.data.AUTOTUNE)
# dataset = dataset.repeat(2)  # doctest: +SKIP

# list(dataset.as_numpy_iterator())

In [ ]:
# Shuffle the dataset and create batches of the data 

BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE)\
.prefetch(tf.data.AUTOTUNE)

In [ ]:
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)\
.prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b"After watching the trailer I was surprised this movie never made it into theaters, so I ordered the BluRay. I had a great time watching it and have to say that this movie is better than some major animation movies out there. Of course, it has its flaws but I can still really recommend it. The animation is well done, very entertaining and unique and the story kept me watching it all the way to the end. Some of the backdrops are just drop-dead gorgeous and you can see the French talent behind it. I thought that Forest Whitaker's performance feels a bit lifeless but that is how the character Lian-Chu is depicted in this movie. So overall, thumbs up, I liked it a lot and I hope it is successful enough for all the studios involved to continue making great movies like this. I would recommend to give it a chance and be surprised how great a movie can be with such a small budget. Hektor alone is worth watching the movie since some of his moments are Stitch-like hilarious."
 b'Get th

In [ ]:
VOCAB_SIZE = 1000

encoder = TextVectorization(max_tokens = VOCAB_SIZE)
 
encoder.adapt(train_dataset.map(lambda text, label: text)) # returns just the text

In [ ]:
# x = train_dataset.map(lambda text, label: text)
# for i in list(x.take(2).as_numpy_iterator()):
#   print(i[:2])

In [ ]:
# To make the data indexable

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[101, 147,   2, ...,   0,   0,   0],
       [ 76,  11,  20, ...,  16, 123,   1],
       [  2, 275, 134, ...,   0,   0,   0]])

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()


Original:  b"After watching the trailer I was surprised this movie never made it into theaters, so I ordered the BluRay. I had a great time watching it and have to say that this movie is better than some major animation movies out there. Of course, it has its flaws but I can still really recommend it. The animation is well done, very entertaining and unique and the story kept me watching it all the way to the end. Some of the backdrops are just drop-dead gorgeous and you can see the French talent behind it. I thought that Forest Whitaker's performance feels a bit lifeless but that is how the character Lian-Chu is depicted in this movie. So overall, thumbs up, I liked it a lot and I hope it is successful enough for all the studios involved to continue making great movies like this. I would recommend to give it a chance and be surprised how great a movie can be with such a small budget. Hektor alone is worth watching the movie since some of his moments are Stitch-like hilarious."
Round-t

In [ ]:
T = len(encoder.get_vocabulary())
T

1000

In [ ]:
encoder

In [ ]:
model = tf.keras.Sequential([
    encoder,
    Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    Bidirectional(LSTM(64,  return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)


Epoch 1/10
391/391 [==============================] - 1155s 3s/step - loss: 0.6806 - accuracy: 0.5155 - val_loss: 0.4217 - val_accuracy: 0.8094
Epoch 2/10
391/391 [==============================] - 1123s 3s/step - loss: 0.4090 - accuracy: 0.8188 - val_loss: 0.3751 - val_accuracy: 0.8396
Epoch 3/10
391/391 [==============================] - 1127s 3s/step - loss: 0.3454 - accuracy: 0.8532 - val_loss: 0.3270 - val_accuracy: 0.8396
Epoch 4/10
391/391 [==============================] - 1129s 3s/step - loss: 0.3240 - accuracy: 0.8592 - val_loss: 0.3179 - val_accuracy: 0.8578
Epoch 5/10
391/391 [==============================] - 1129s 3s/step - loss: 0.3138 - accuracy: 0.8668 - val_loss: 0.3368 - val_accuracy: 0.8552
Epoch 6/10
391/391 [==============================] - 1125s 3s/step - loss: 0.3081 - accuracy: 0.8676 - val_loss: 0.3137 - val_accuracy: 0.8615
Epoch 7/10
391/391 [==============================] - 1130s 3s/step - loss: 0.3052 - accuracy: 0.8673 - val_loss: 0.3027 - val_accuracy:

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))


391/391 [==============================] - 225s 573ms/step - loss: 0.3434 - accuracy: 0.8624
Test Loss: 0.34335651993751526
Test Accuracy: 0.8624399900436401


In [ ]:
# predict on a sample text.

sample_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)


[[-1.9851784]]
